In [15]:
import ee
import geemap
ee.Initialize()

In [16]:
import requests
import pandas as pd
import json
import time

In [17]:
GEOCODE = '4318309'
url = f'http://servicodados.ibge.gov.br/api/v3/malhas/municipios/{GEOCODE}?formato=application/vnd.geo+json'
response = requests.get(url)
geo = response.json();
ee_object = geemap.geojson_to_ee(geo)
geometry = ee_object.geometry()

In [2]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)

# Define a region in which to generate a sample of the input.
region = geometry = ee.Geometry.Polygon(
        [[[-46.31633300781251, -12.379228237680417],
          [-45.31108398437501, -12.298734131305821],
          [-45.30009765625001, -11.589339623793398],
          [-46.33830566406251, -11.605482676116234]]])

Map

#Draw the region polygon

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [35]:
Map.draw_features
roi = ee.FeatureCollection(Map.draw_features)
geometry = roi.geometry()
Map

Map(bottom=35226.0, center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [13]:
Map.addLayer(ee_object)
Map

Map(bottom=35226.0, center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [18]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':10000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':10000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [19]:
import requests
import pandas as pd
import json
import time
#import os

#out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')


In [20]:
stats = list();

In [ ]:
#Dividir o processamento das imagens de acordo com o ano para não sobrecarregar o sistema
ano = list(range(2000, 2022))

string_ints = [str(int) for int in ano]
#stats = list();
    
for i in (string_ints):
    
    print (i)
    #Filtrar a territorialidade 
    #4 principais meses de safra
    #filter start and end date   
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( i + '-01-01', i +'-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
        
    #Pegar a propriedade que nos interessa em cada imagem - o EVI_Cluster Calculado 
    a = mcollection.aggregate_array('EVI_Cluster')
    
    
    #10s de soneca pra o GEE não derrubar as requests
    time.sleep(10)
    # Fazer a informação passar do servidor do google para o jupyter
    x = a.getInfo()
    #Salvar cada ano
    stats.append(x)    
    

2000
2001
2002


In [ ]:
stats

In [8]:
df = pd.DataFrame({'lin':stats})

In [9]:
#Separate comma
#df = df.apply(lambda x:pd.Series(x))

In [10]:
df = pd.DataFrame(stats, columns = ['EVI1', 'EVI2','EVI3','EVI4','EVI5','EVI6','EVI7','EVI8'])

In [11]:
df

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842,NaN,NaN,NaN
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787


In [12]:
#Reshape da 1a linha - Imagens do MODIS a partir de Mar/2000
new_order =  ['EVI6', 'EVI7', 'EVI8','EVI1','EVI2','EVI3','EVI4','EVI5'] # specify new order of the third row
i = 0 # specify row number
df.loc[0] = df.loc[0, new_order].values

In [13]:
#Corrigir os Nans
#Features dos EVIS pronta
df.fillna(0)


,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,0.000000,0.000000,0.000000,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787


In [14]:
GEOCODE = '2919553'


In [15]:
url = f'https://apiprevmet3.inmet.gov.br/estacao/proxima/{GEOCODE}'
response = requests.get(url).json()
response

{'estacao': {'UF': 'TO',
  'CODIGO': 'A038',
  'LONGITUDE': '-46.84722221',
  'REGIAO': 'N',
  'DISTANCIA_EM_KM': '115',
  'NOME': 'DIANOPOLIS',
  'LATITUDE': '-11.59444444',
  'GEOCODE': '1707009'},
 'dados': {'DC_NOME': 'DIANOPOLIS',
  'PRE_INS': '934.7',
  'TEM_SEN': '25.7',
  'VL_LATITUDE': '-11.59444444',
  'PRE_MAX': '934.7',
  'UF': 'TO',
  'RAD_GLO': '2285.817',
  'PTO_INS': '9.6',
  'TEM_MIN': '27.8',
  'VL_LONGITUDE': '-46.84722221',
  'UMD_MIN': '29',
  'PTO_MAX': '11.3',
  'VEN_DIR': '123',
  'DT_MEDICAO': '2021-07-28',
  'CHUVA': '0',
  'PRE_MIN': '934.4',
  'UMD_MAX': '35',
  'VEN_VEL': '2.8',
  'PTO_MIN': '8.7',
  'TEM_MAX': '29.1',
  'VEN_RAJ': '4.8',
  'TEM_INS': '28.4',
  'UMD_INS': '31',
  'CD_ESTACAO': 'A038',
  'HR_MEDICAO': '1300'}}

In [16]:
estacao = response['dados']['CD_ESTACAO']

In [17]:
new_url = f'https://apitempo.inmet.gov.br/estacao/diaria/1999-11-01/2021-04-30/{estacao}'
json_response = requests.get(new_url).json()
dados_estacao = pd.DataFrame(json_response)
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE
0,None,1999-11-01,DIANOPOLIS,None,None,None,-11.59444444,None,None,TO,None,A038,-46.84722221
1,None,1999-11-02,DIANOPOLIS,None,None,None,-11.59444444,None,None,TO,None,A038,-46.84722221
2,None,1999-11-03,DIANOPOLIS,None,None,None,-11.59444444,None,None,TO,None,A038,-46.84722221
3,None,1999-11-04,DIANOPOLIS,None,None,None,-11.59444444,None,None,TO,None,A038,-46.84722221
4,None,1999-11-05,DIANOPOLIS,None,None,None,-11.59444444,None,None,TO,None,A038,-46.84722221
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,86.9,2021-04-26,DIANOPOLIS,57,22.9,None,-11.59444444,18.6,30.3,TO,0.8,A038,-46.84722221
7848,88.5,2021-04-27,DIANOPOLIS,58,22.6,None,-11.59444444,19.3,28.9,TO,1.7,A038,-46.84722221
7849,83.8,2021-04-28,DIANOPOLIS,69,23.7,None,-11.59444444,21.4,28.1,TO,0.4,A038,-46.84722221
7850,83.3,2021-04-29,DIANOPOLIS,52,23.9,None,-11.59444444,20,31.6,TO,1.4,A038,-46.84722221


In [18]:
dados_estacao['CHUVA'] = dados_estacao['CHUVA'].astype(float)
dados_estacao['DT_MEDICAO'] = pd.to_datetime(dados_estacao['DT_MEDICAO'])
dados_estacao['ANO'] = dados_estacao['DT_MEDICAO'].dt.year
dados_estacao['MES'] = dados_estacao['DT_MEDICAO'].dt.month
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE,ANO,MES
0,None,1999-11-01,DIANOPOLIS,None,None,NaN,-11.59444444,None,None,TO,None,A038,-46.84722221,1999,11
1,None,1999-11-02,DIANOPOLIS,None,None,NaN,-11.59444444,None,None,TO,None,A038,-46.84722221,1999,11
2,None,1999-11-03,DIANOPOLIS,None,None,NaN,-11.59444444,None,None,TO,None,A038,-46.84722221,1999,11
3,None,1999-11-04,DIANOPOLIS,None,None,NaN,-11.59444444,None,None,TO,None,A038,-46.84722221,1999,11
4,None,1999-11-05,DIANOPOLIS,None,None,NaN,-11.59444444,None,None,TO,None,A038,-46.84722221,1999,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,86.9,2021-04-26,DIANOPOLIS,57,22.9,NaN,-11.59444444,18.6,30.3,TO,0.8,A038,-46.84722221,2021,4
7848,88.5,2021-04-27,DIANOPOLIS,58,22.6,NaN,-11.59444444,19.3,28.9,TO,1.7,A038,-46.84722221,2021,4
7849,83.8,2021-04-28,DIANOPOLIS,69,23.7,NaN,-11.59444444,21.4,28.1,TO,0.4,A038,-46.84722221,2021,4
7850,83.3,2021-04-29,DIANOPOLIS,52,23.9,NaN,-11.59444444,20,31.6,TO,1.4,A038,-46.84722221,2021,4


In [19]:
b  = dados_estacao.groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [20]:
mask = (dados_estacao['MES'] <= 4) | (dados_estacao['MES'] >= 11)  
b = dados_estacao.loc[mask].groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [21]:
chuva = b.values.reshape(22,6)
df2 = pd.DataFrame(chuva, columns = ['C_NOV', 'C_DEZ','C_JAN','C_FEV','C_MAR','C_ABR'])
df2

,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,261.4,320.8,367.4,231.4,273.8,337.4


In [22]:
concat_data = pd.concat([df,df2],axis=1).fillna(0)

In [23]:
concat_data

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,0.000000,0.000000,0.000000,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842,0.0,0.0,0.0,0.0,0.0,0.0
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507,0.0,0.0,0.0,0.0,0.0,0.0
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139,0.0,0.0,0.0,0.0,0.0,0.0
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042,0.0,0.0,0.0,0.0,0.0,0.0
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827,0.0,0.0,0.0,0.0,0.0,0.0
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751,0.0,0.0,0.0,0.0,0.0,0.0
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018,0.0,0.0,0.0,0.0,0.0,0.0
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769,0.0,0.0,0.0,0.0,0.0,0.0
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017,0.0,0.0,0.0,0.0,0.0,0.0
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787,261.4,320.8,367.4,231.4,273.8,337.4


In [24]:
# Api PAM SIDRA/ https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124

In [25]:
url3 = 'https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124'
json_response = requests.get(url3).json()
produtividade = pd.DataFrame(json_response)
Y = produtividade['V'].iloc[1:].values.ravel()
Y[0] = Y[1];

In [26]:
#Produtividade de Soja por ano
df3 = pd.DataFrame(Y)


In [27]:
import sklearn

In [28]:
#Divisão do modelo para teste e para treinamento
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(concat_data.iloc[0:20,0:4].fillna(0), Y, test_size=(0.20))

In [79]:
X_train2 = df.iloc[1:21,2:3].fillna(0)
X_test2 =  df.iloc[21:22,2:3]
y_train2 = Y[1:20]
y_test2 = Y[19:20]

In [30]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [31]:
#Importamos o classificador KNN e aplicamos o modelo com k vizinhos com base em X_train e Y-train

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(X_train_minmax, y_train)

KNeighborsClassifier(n_neighbors=10)

In [81]:
import numpy as np
y_train2 = np.append(y_train2,'3950')

In [33]:
#Métricas de acurácia do sklearn

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined

In [34]:
#Predição para a safra 2006
classifier.predict(df.iloc[1:18,:])

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 8 while Y.shape[1] == 4

In [35]:
sklearn.preprocessing.normalize(X_train2)

array([[0.66485652, 0.74697109],
       [0.71385452, 0.70029402],
       [0.6960587 , 0.71798488],
       [0.66097138, 0.75041111],
       [0.65172917, 0.75845177],
       [0.62068591, 0.78405931],
       [0.70436857, 0.70983443],
       [0.69095348, 0.72289922],
       [0.67405864, 0.73867784],
       [0.71962883, 0.69435895],
       [0.73490195, 0.67817337],
       [0.72666094, 0.68699627],
       [0.69674456, 0.71731933],
       [0.69069962, 0.72314178],
       [0.65848066, 0.75259765],
       [0.701529  , 0.71264091],
       [0.677602  , 0.7354288 ],
       [0.69609244, 0.71795217],
       [0.7386383 , 0.67410197],
       [0.72168526, 0.69222134]])

In [85]:
# importing module
from sklearn.linear_model import LinearRegression
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(X_train2,y_train2)

LinearRegression()

In [86]:
y_prediction =  LR.predict(X_train2)

In [87]:
y_train2

array(['2040', '1829', '1829', '2880', '2759', '2280', '2700', '3036',
       '2550', '3060', '3360', '3480', '2280', '2580', '3240', '2100',
       '3413', '4159', '3818', '3950'], dtype=object)

In [88]:
y_prediction

array([2431.69411311, 2932.48077794, 2588.02673407, 3004.75024168,
       2823.67444295, 2627.83189874, 2810.5112907 , 2824.92332512,
       3165.99697926, 2890.76308392, 2819.98074642, 2635.14034   ,
       2871.44375939, 2556.07815703, 3029.68002422, 3165.40036433,
       2948.51504163, 3521.77607983, 2593.81264243, 3100.51995723])

In [53]:
X_test2

,EVI2,EVI3
21,8289.355512,8378.505445


In [64]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_train2,y_prediction)
print("r2 socre is ",score)
print("mean_sqrd_error is==",mean_squared_error(y_train2,y_prediction))


r2 socre is  0.26870263051025556
mean_sqrd_error is== 336661.9053724002


In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import numpy as np

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=3)
y_rbf = svr_rbf.fit(X_train2, y_train2).predict(X_test2)
y_lin = svr_lin.fit(X_train2, y_train2).predict(X_test2)
y_poly = svr_poly.fit(X_train2, y_train2).predict(X_test2)


In [93]:
print(y_lin)


[5469.88196539]


In [82]:
from sklearn.naive_bayes import BernoulliNB

gnb = BernoulliNB()

y_pred = gnb.fit(X_train2,y_train2).predict(X_test2)

In [84]:
y_pred

array(['1829'], dtype='<U4')

In [ ]:
Y_predict




In [ ]:
y_poly

In [ ]:
#Overfitted total kkk
Y_predict

In [ ]:
x_Test3 = df.iloc[19:22,1:3]

In [ ]:
x_Test3

In [ ]:
y_poly = svr_poly.fit(X_train2, y_train2).predict(x_Test3)

In [ ]:
y_poly

In [61]:
y_train2

array(['2040', '1829', '1829', '2880', '2759', '2280', '2700', '3036',
       '2550', '3060', '3360', '3480', '2280', '2580', '3240', '2100',
       '3413', '4159', '3818', 3950], dtype=object)

In [75]:
X_test2

,EVI2,EVI3
21,8289.355512,8378.505445
